In [3]:
import pandas as pd
import numpy as np
from numpy.random import *
import matplotlib.pyplot as plt
from scipy.special import comb
import math

### Value

In [4]:
def value(N, u, d, pi, default_x):
    val_arr = []
    recursive = []
    for l in range(2**(N+1)):
        for m in range(2):
            recursive.append(l)
    for l in range(N+1):
        val_arr.append([])
        val_arr[l] = [0 for m in range(2**l)]
        if l==0:
            val_arr[l][l]=default_x
        else:
            for n in range(len(val_arr[l])):
                if (n+1)%2!=0:
                    val_arr[l][n] = val_arr[l-1][recursive[n]]*(1+pi*(u-1))
                if (n+1)%2==0:
                    val_arr[l][n] = val_arr[l-1][recursive[n]]*(1+pi*(d-1))

    return val_arr[N]

In [5]:
N=10
num = [ i for i in range(2**(N+1))]
stock = [[ [] for i in range(N+1) ] for j in range(N+1)]
for i in range(N+1):
        for j in range(N+1):
            stock[i][0] =num[(2**(i+1))-1::-1]

### Backward (+$\epsilon$)

In [6]:
def backward(N, theta, u, d, p, price_x, default_x):
    
    seed(123)
    
    gamma = (1-1/theta)
    K = (p*(u-1)/((1-p)*(1-d)))**(1/(1-gamma))
    pi = (K-1)/(u-1+K*(1-d))
    M = p*(1+pi*(u-1))**(1-(1/theta))+(1-p)*(1+pi*(d-1))**(1-(1/theta))

    num = [ i for i in range(2**(N+1))]
    stock = [[ [] for i in range(N+1) ] for j in range(N+1)]
    
     
    for i in range(N+1):
        for j in range(N+1):
            #index
            stock[i][0] = i
            #node_num
            stock[i][1] = num[0:i+1]
            #up_times
            stock[i][2] = num[(2**(i))-1::-1]
            #down_times
            stock[i][3] = num[0:(2**i)]
            #theta
            stock[i][4] = theta
            #u
            stock[i][5] = u
            #+(0.1*random()) if i>1 else u if i==1  else 0
            #d
            stock[i][6] = d
            #+(0.1*random()) if i>1 else d if i==1  else 0
            #prob
            stock[i][7] = p
            #M
            stock[i][8] = stock[i][7]*(1+pi*(stock[i][5]-1))**(1-(1/stock[i][4]))+(1-stock[i][7])*(1+pi*(stock[i][6]-1))**(1-(1/stock[i][4]))
            #value
            stock[i][9] = value(i, stock[i][5], stock[i][6], pi, default_x)
            #Wealth
            stock[i][10] = list(map(lambda x: (stock[i][8]**((10)-i))*((1-(1/stock[i][4]))**(-1))*(x)**(1-(1/stock[i][4])), stock[i][9]))
    
            #Utility
            stock[i][11] = list(map(lambda x: (1-(1/stock[i][4]))**(-1)*x**(1-(1/stock[i][4])), stock[i][10]))
            
            #Exp_value
            stock[i][12] = list(map(lambda  l, m, n : comb((2**i)-1, l, exact=True)if (l!=0 and m!=0) \
                                    #*(p**l)*((1-p)**m)*n if (l!=0 and m!=0) \
                                 else (p**l)*n if m==0 \
                                 else ((1-p)**m)*n if l==0 \
                                 else nan \
                                , stock[i][2], stock[i][3], stock[i][9]))
            #Wn
            stock[i][13] = list(map(lambda  l, m, n : comb(i, l, exact=True)*(p**l)*((1-p)**m)*n if (l!=0 and m!=0) \
                                 else (p**l)*n if m==0 \
                                 else ((1-p)**m)*n if l==0 \
                                 else nan \
                                , stock[i][2], stock[i][3], stock[i][10]))
    df = pd.DataFrame(stock)
    df = df.set_index(0)
    df.columns = ["node_num", "up_times", "down_times", "theta", "u", "d", "prob", "M", "value", "wealth", "utility", "Exp_val", "Wn"]
    
    
    return df#np.sum(df["Exp_val"][10]), df["wealth"][0][0], math.pow(df["wealth"][0][0]*(1-(1/theta)), 1/(1-(1/theta)))

In [7]:
df = backward(13, 1.2, 1.2, 0.8, 0.4, 100, 500)

KeyboardInterrupt: 

In [ ]:
df

### Forward $(+\epsilon)$

In [3]:
def delta (theta, a, b, c):
    return ((1+b)/((c**(theta))*((a**(-theta))+b)))

In [41]:
def forward(N, theta, u, d, p, default_x):

    seed(123)

    q = (1-d)/(u-d)
    a = ((1-p)/p)*(q/(1-q))
    b = (1-q)/q
    c = (1-p)/(1-q)

    num = [ i for i in range(N+1)]
    stock = [[ [] for i in range(N+1) ] for j in range(N+1)]
    
    pi = (delta(theta, a, b, c)*((p/q)**(theta))-1)/(u-1)
    M = p*(1+pi*(u-1))**(1-(1/theta))+(p-1)*(1+pi*(d-1))**(1-(1/theta))
    val = value(N+1, u, d, pi, default_x)
    
    for i in range(N+1):
        for j in range(N+1):
            #index
            stock[i][0] = i
            #node_num
            stock[i][1] = num[0:i+1]
            #up_times
            stock[i][2] = num[i::-1]
            #down_times
            stock[i][3] = num[0:i+1]
            #theta
            stock[i][4] = theta
            #u
            stock[i][5] = u+(0.1*random()) if i>1 else u if i==1  else 0
            #d
            stock[i][6] = d+(0.1*random()) if i>1 else d if i==1  else 0
            #p
            stock[i][7] = p
            #q
            stock[i][8] = (1-stock[i][6])/(stock[i][5]-stock[i][6]) if i>1 else q if i==1  else 0
            #a
            stock[i][9] = ((1-stock[i][7])/stock[i][7])*(stock[i][8]/(1-stock[i][8])) if i>1 else a if i==1 else 0
            #b
            stock[i][10] = ((1-stock[i][8])/stock[i][8]) if i>1 else b if i==1 else 0
            #c
            stock[i][11] = ((1-stock[i][7])/(1-stock[i][8])) if i>1 else c if i==1 else 0
            #pi
            stock[i][12] = (delta(theta, stock[i][9], stock[i][10], stock[i][11])*((stock[i][7]/stock[i][8])**(theta))-1)/(stock[i][5]-1) if i>1 else pi if i==1 else 0
            #price
            #stock[i][10] = list(map(lambda x, y: default_x if x==0 and y==0 else (default_x*x)+(default_x*y) if x==0 or y==0 else (default_x*x)*(default_x*y), stock[i][8], stock[i][9]))            
            #value
            stock[i][13] = value(i, stock[i][5], stock[i][6], stock[i][12], default_x)
            #Wealth
            #プログラム上、ここのNを10にして10期間にしている
            stock[i][14] = list(map(lambda x: (M**(10-i))*((1/(1-theta))**(-1))*(x)**(1-(1/theta)), stock[i][13]))
            #U_0
            stock[i][15] = list(map(lambda x: ((1-(1/theta))**(-1))*(x**(1-(1/theta))), stock[i][13]))
            #I_0
            #stock[i][10] = list(map(lambda x: (x**(-1/theta))**(-1), stock[i][8]))
            #I_1
            #stock[i][11] = list(map(lambda x: (1+b)/((c**theta)*((a**(-theta))+b)*x, stock[i][10]))
            #U_1
            stock[i][16] = list(map(lambda x: (((1+b)/((c**(theta))*((a**(-theta))+b)))**(1/theta))*x, stock[i][15]))
            #pi
            #stock[i][14] = list(map(lambda x: ((((1+b)/((c**theta)*((a**(-theta))+b)))*((p/q)**theta)-1)/(u-1))*x, stock[i][13]))
            #Exp[U_1]
            stock[i][17] = list(map(lambda  l, m, n : comb(i, l, exact=True)*(p**l)*((1-p)**m)*n if (l!=0 and m!=0) \
                                 else (p**l)*n if m==0 \
                                 else ((1-p)**m)*n if l==0 \
                                 else nan \
                                 , stock[i][2], stock[i][3], stock[i][16]))
           
            
    df = pd.DataFrame(stock)
    df = df.set_index(0)
    df.columns = ["node_num", "up_times", "down_times", "theta", "u", "d", "p", "q", "a", "b", "c", "pi", "value", "wealth", "U_0", "U_1", "Exp_U1"]
    
    return df
    

In [25]:
forward(17, 2.0, 1.2, 0.8, 0.4, 500)

,node_num,up_times,down_times,theta,u,d,p,q,a,b,c,pi,value,wealth,U_0,U_1,Exp_U1
0,,,,,,,,,,,,,,,,,
0,[0],[0],[0],2.0,0.000000,0.000000,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,[500],[-0.0019271619793530532],[44.721359549995796],[43.85290096535146],[43.85290096535146]
1,"[0, 1]","[1, 0]","[0, 1]",2.0,1.200000,0.800000,0.4,0.500000,1.500000,1.000000,1.200000,-1.923077,"[307.69230769230774, 692.3076923076923]","[0.003854323958706106, 0.005781485938059159]","[35.08232077228117, 52.623481158421754]","[34.40104580768907, 51.60156871153361]","[13.76041832307563, 30.960941226920163]"
2,"[0, 1, 2]","[2, 1, 0]","[0, 1, 2]",2.0,1.242583,0.831226,0.4,0.410286,1.043605,1.437326,1.017442,-0.205812,"[451.31992677478667, 491.53749898755234, 491.5...","[-0.01190114723040956, -0.012420094525301633, ...","[42.48858325596591, 44.341289967142465, 44.341...","[41.66348367828496, 43.48021207698758, 43.4802...","[6.666157388525595, 20.870501796954038, 15.652..."
3,"[0, 1, 2, 3]","[3, 2, 1, 0]","[0, 1, 2, 3]",2.0,1.209571,0.888533,0.4,0.347208,0.797824,1.880114,0.919130,1.484401,"[1126.8481080886281, 717.2649056171382, 717.26...","[0.0479441851329145, 0.03825102196517717, 0.03...","[67.13711665207639, 53.56360352392801, 53.5636...","[65.83335921063319, 52.52343453593387, 52.5234...","[4.213334989480525, 15.126749146348958, 22.690..."
4,"[0, 1, 2, 3, 4]","[4, 3, 2, 1, 0]","[0, 1, 2, 3, 4]",2.0,1.284651,0.855326,0.4,0.336981,0.762378,1.967528,0.904951,1.350384,"[1836.5415566584622, 1067.4353337807404, 1067....","[-0.15604875568629786, -0.11896815250489674, -...","[85.70977906069908, 65.34325776331451, 65.3432...","[84.04535306465978, 64.0743358493969, 64.07433...","[2.1515610384552906, 9.841817986467367, 22.144..."
5,"[0, 1, 2, 3, 4, 5]","[5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5]",2.0,1.292758,0.856900,0.4,0.328317,0.733196,2.045836,0.893279,1.538948,"[3210.8402108337186, 1726.0740078217666, 1726....","[0.5260487040103635, 0.3856970478596833, 0.385...","[113.32855263937184, 83.09209367495241, 83.092...","[111.12778872218924, 81.47850136035807, 81.478...","[1.137948556515218, 6.2575489044755015, 18.772..."
6,"[0, 1, 2, 3, 4, 5, 6]","[6, 5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5, 6]",2.0,1.239938,0.842431,0.4,0.396394,0.985063,1.522745,0.994025,0.075950,"[557.2220936805181, 540.7002019209142, 540.700...","[-0.5587112941800348, -0.5503659442145683, -0....","[47.211104358212935, 46.50592228613101, 46.505...","[46.29429661169098, 45.60280869896376, 45.6028...","[0.18962143892148633, 1.6811019398786005, 6.30..."
7,"[0, 1, 2, 3, 4, 5, 6, 7]","[7, 6, 5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5, 6, 7]",2.0,1.241157,0.848103,0.4,0.386454,0.944804,1.587631,0.977922,0.292360,"[805.5455201978527, 719.0741264196039, 719.074...","[1.7126751974853243, 1.6181426175977605, 1.618...","[56.76426764075628, 53.63111508889607, 53.6311...","[55.661943918273025, 52.589635071927205, 52.58...","[0.09119652891569856, 0.9047100100693783, 4.07..."
8,"[0, 1, 2, 3, 4, 5, 6, 7, 8]","[8, 7, 6, 5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5, 6, 7, 8]",2.0,1.220825,0.844337,0.4,0.413461,1.057376,1.418606,1.022950,-0.293236,"[292.669516083935, 327.2173128894618, 327.2173...","[-2.6319363856448548, -2.782946005720645, -2.7...","[34.21517301338311, 36.178298074368385, 36.178...","[33.55073747234494, 35.475739971111665, 35.475...","[0.021987811309875992, 0.278992571369613, 1.46..."


In [12]:
def epsilon(N, theta, u, d, p, default_x):

    seed(123)

    q = (1-d)/(u-d)
    a = ((1-p)/p)*(q/(1-q))
    b = (1-q)/q
    c = (1-p)/(1-q)

    num = [ i for i in range(N+1)]
    stock = [[ [] for i in range(N+1) ] for j in range(N+1)]
    
    gamma = (1-1/theta)
    K = (p*(u-1)/((1-p)*(1-d)))**(1/(1-gamma))
    pi_backward = (K-1)/(u-1+K*(1-d))

    #M = p*(1+pi*(u-1))**(1-(1/theta))+(p-1)*(1+pi*(d-1))**(1-(1/theta))
    #val = value(N+1, u, d, pi, default_x)
    
    for i in range(N+1):
        for j in range(N+1):
            #index
            stock[i][0] = i
            #node_num
            stock[i][1] = num[0:i+1]
            #up_times
            stock[i][2] = num[i::-1]
            #down_times
            stock[i][3] = num[0:i+1]
            #theta
            stock[i][4] = theta
            #u
            stock[i][5] = u
            #+(0.1*random()) if i>1 else u if i==1  else 0
            #d
            stock[i][6] = d
            #+(0.1*random()) if i>1 else d if i==1  else 0
            #p
            stock[i][7] = p
            #q
            stock[i][8] = (1-stock[i][6])/(stock[i][5]-stock[i][6]) if i>1 else q if i==1  else 0
            #a
            stock[i][9] = ((1-stock[i][7])/stock[i][7])*(stock[i][8]/(1-stock[i][8])) if i>1 else a if i==1 else 0
            #b
            stock[i][10] = ((1-stock[i][8])/stock[i][8]) if i>1 else b if i==1 else 0
            #c
            stock[i][11] = ((1-stock[i][7])/(1-stock[i][8])) if i>1 else c if i==1 else 0
            #pi_forward
            stock[i][12] = (delta(stock[i][4], stock[i][9], stock[i][10], stock[i][11])*((stock[i][7]/stock[i][8])**(stock[i][4]))-1)/(stock[i][5]-1) if i>1 else pi_backward if i==1 else 0
            #pi_backward
            stock[i][13] =  pi_backward
            #price
            #stock[i][10] = list(map(lambda x, y: default_x if x==0 and y==0 else (default_x*x)+(default_x*y) if x==0 or y==0 else (default_x*x)*(default_x*y), stock[i][8], stock[i][9]))            
            #value_forward
            stock[i][14] = value(i, stock[i][5], stock[i][6], stock[i][12], default_x)
            #value_backward
            stock[i][15] = value(i, stock[i][5], stock[i][6], stock[i][13], default_x)
            #Wealth_forward
            #プログラム上、ここのNを10にして10期間にしている
            M = stock[i][7]*(1+stock[i][12]*(stock[i][5]-1))**(1-(1/stock[i][4]))+(stock[i][7]-1)*(1+stock[i][12]*(stock[i][6]-1))**(1-(1/stock[i][4]))
            stock[i][16] = list(map(lambda x: (M**(10-i))*((1/(1-stock[i][4]))**(-1))*(x)**(1-(1/stock[i][4])), stock[i][14]))
            
            #Wealth_backward
            M = stock[i][7]*(1+stock[i][13]*(stock[i][5]-1))**(1-(1/stock[i][4]))+(stock[i][7]-1)*(1+stock[i][13]*(stock[i][6]-1))**(1-(1/stock[i][4]))
            stock[i][17] = list(map(lambda x: (M**(10-i))*((1/(1-stock[i][4]))**(-1))*(x)**(1-(1/stock[i][4])), stock[i][15]))
            
            #U_0_forward
            stock[i][18] = list(map(lambda x: ((1-(1/stock[i][4]))**(-1))*(x**(1-(1/stock[i][4]))), stock[i][14]))
            #U_1_forward
            stock[i][19] = list(map(lambda x: (((1+b)/((c**(theta))*((a**(-theta))+b)))**(1/theta))*x, stock[i][18]))
            #stock[i][19] = list(map(lambda x: (((1+stock[i][10])/((stock[i][11]**(stock[i][4]))*((stock[i][9]**(-stock[i][4]))+stock[i][10])))**(1/stock[i][4]))*x, stock[i][18]))
            
            #U_0_backward
            stock[i][20] = list(map(lambda x: ((1-(1/theta))**(-1))*(x**(1-(1/theta))), stock[i][15]))
            #I_0
            #stock[i][10] = list(map(lambda x: (x**(-1/theta))**(-1), stock[i][8]))
            #I_1
            #stock[i][11] = list(map(lambda x: (1+b)/((c**theta)*((a**(-theta))+b)*x, stock[i][10]))
            
            #pi
            #stock[i][14] = list(map(lambda x: ((((1+b)/((c**theta)*((a**(-theta))+b)))*((p/q)**theta)-1)/(u-1))*x, stock[i][13]))
            #Exp[U_1]_forward
            stock[i][21] = list(map(lambda  l, m, n : comb(i, l, exact=True)*(stock[i][7]**l)*((1-stock[i][7])**m)*n if (l!=0 and m!=0) \
                                 else (stock[i][7]**l)*n if m==0 \
                                 else ((1-stock[i][7])**m)*n if l==0 \
                                 else nan \
                                 , stock[i][2], stock[i][3], stock[i][19]))
            
            #Exp[U_1]_backward
            stock[i][22] = list(map(lambda  l, m, n : comb(i, l, exact=True)*(stock[i][7]**l)*((1-stock[i][7])**m)*n if (l!=0 and m!=0) \
                                 else (stock[i][7]**l)*n if m==0 \
                                 else ((1-stock[i][7])**m)*n if l==0 \
                                 else nan \
                                 , stock[i][2], stock[i][3], stock[i][20]))
           
            
    df = pd.DataFrame(stock)
    df = df.set_index(0)
    df.columns = ["node_num", "up_times", "down_times", "theta", "u", "d", "p", "q", "a", "b", "c", "pi_forward", "pi_backward", "value_forward", "value_backward", "wealth_forward", "wealth_backward", "U_0_forward", "U_1_forward", "U_0_backward", "Exp_U1_forward", "Exp_U1_backward"]
    
    return df
    

In [13]:
epsilon(22, 1.2, 1.2, 0.8, 0.4, 500)

,node_num,up_times,down_times,theta,u,d,p,q,a,b,...,pi_backward,value_forward,value_backward,wealth_forward,wealth_backward,U_0_forward,U_1_forward,U_0_backward,Exp_U1_forward,Exp_U1_backward
0,,,,,,,,,,,,,,,,,,,,,
0,[0],[0],[0],1.2,1.2,0.8,0.4,0.0,0.0,0.0,...,-1.192953,[500],[500],[-5.769767145160363e-08],[-3.505946971047279e-07],[16.903614683087042],[16.83597002276827],[16.903614683087042],[16.83597002276827],[16.903614683087042]
1,"[0, 1]","[1, 0]","[0, 1]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[380.7047188569014, 619.2952811430985]","[380.7047188569014, 619.2952811430985]","[1.398557578252725e-06, 1.5166962140096965e-06]","[1.398557578252725e-06, 1.5166962140096965e-06]","[16.15285496905992, 17.517315238195955]","[16.088214688975107, 17.447214679161664]","[16.15285496905992, 17.517315238195955]","[6.4352858755900435, 10.468328807496999]","[6.461141987623968, 10.510389142917573]"
2,"[0, 1, 2]","[2, 1, 0]","[0, 1, 2]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[289.87216591982474, 471.5372717939781, 471.53...","[289.87216591982474, 471.5372717939781, 471.53...","[-5.578987120572027e-06, -6.050254044135894e-0...","[-5.578987120572027e-06, -6.050254044135894e-0...","[15.435439611182257, 16.73929853435394, 16.739...","[15.373670274330651, 16.67231143221277, 16.672...","[15.435439611182257, 16.73929853435394, 16.739...","[2.4597872438929045, 8.002709487462129, 6.0020...","[2.469670337789162, 8.03486329648989, 6.026147..."
3,"[0, 1, 2, 3]","[3, 2, 1, 0]","[0, 1, 2, 3]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[220.71140286189592, 359.0329289777535, 359.03...","[220.71140286189592, 359.0329289777535, 359.03...","[2.225514185150275e-05, 2.4135073101955213e-05...","[2.225514185150275e-05, 2.4135073101955213e-05...","[14.749887648147462, 15.995836782753534, 15.99...","[14.690861744019553, 15.931824855957395, 15.93...","[14.749887648147462, 15.995836782753534, 15.99...","[0.9402151516172517, 4.58836555851573, 6.88254...","[0.9439928094814378, 4.606800993433018, 6.9102..."
4,"[0, 1, 2, 3, 4]","[4, 3, 2, 1, 0]","[0, 1, 2, 3, 4]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[168.0517451501008, 273.371060573691, 273.3710...","[168.0517451501008, 273.371060573691, 273.3710...","[-8.877800362796423e-05, -9.627723883781408e-0...","[-8.877800362796423e-05, -9.627723883781408e-0...","[14.094783894289707, 15.285395254489153, 15.28...","[14.038379575647156, 15.224226363147666, 15.22...","[14.094783894289707, 15.285395254489153, 15.28...","[0.35938251713656727, 2.338441169379482, 5.261...","[0.36082646769381654, 2.3478367110895344, 5.28..."
5,"[0, 1, 2, 3, 4, 5]","[5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[127.95618478156155, 208.14730551864002, 208.1...","[127.95618478156155, 208.14730551864002, 208.1...","[0.00035414440315664135, 0.0003840596081510347...","[0.00035414440315664135, 0.0003840596081510347...","[13.468776018214616, 14.606507384338292, 14.60...","[13.414876849560862, 14.548055257442272, 14.54...","[13.468776018214616, 14.606507384338292, 14.60...","[0.13736833893950326, 1.1172906437715668, 3.35...","[0.1379202664265177, 1.1217797671171812, 3.365..."
6,"[0, 1, 2, 3, 4, 5, 6]","[6, 5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5, 6]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[97.42704670653224, 158.48532285659084, 158.48...","[97.42704670653224, 158.48532285659084, 158.48...","[-0.0014127177134186893, -0.001532052480873528...","[-0.0014127177134186893, -0.001532052480873528...","[12.870571751180089, 13.957771743329333, 13.95...","[12.819066468402431, 13.901915718089407, 13.90...","[12.870571751180089, 13.957771743329333, 13.95...","[0.05250689625457638, 0.512480221031648, 1.921...","[0.052717861892833666, 0.5145392975460926, 1.9..."
7,"[0, 1, 2, 3, 4, 5, 6, 7]","[7, 6, 5, 4, 3, 2, 1, 0]","[0, 1, 2, 3, 4, 5, 6, 7]",1.2,1.2,0.8,0.4,0.5,1.5,1.0,...,-1.192953,"[74.18187285093713, 120.67222056212735, 120.67...","[74.18187285093713, 120.67222056212735, 120.67...","[0.00563547332674966, 0.006111511740197